In [1]:
# Path & Libraries

import sys
import os
import warnings

sys.path.append(os.path.abspath('dss'))
warnings.filterwarnings('ignore')

import pandas as pd
import pickle
import zipfile

from src.dss_irrigation.mamdani.dss_package import apply_decision_system, reverse_categorize_irrigation, show_figure

In [7]:
# Data Loading & Pre-Processing

zip_path = '../src/dss_irrigation/mamdani/models_ml.zip'
model_name = 'sensor_67_lr.pkl'
with zipfile.ZipFile(zip_path, 'r') as zipf:
    with zipf.open(model_name) as file:
        xgb_model = pickle.load(file)

dss_file_path = '../src/dss_irrigation/mamdani/model_dss.pkl'
with open(dss_file_path, 'rb') as file:
    dss = pickle.load(file)


file_path = '../data/dss_data_rovere.csv'
start_date = '2023-05-01'
end_date = '2023-08-31'
sensor_id = 67

df = pd.read_csv(file_path)
df = df[(df['sensor_id'] == sensor_id) & (df['date'] >= start_date) & (df['date'] <= end_date)]
df.set_index('date', inplace=True)

real_tens_values = df['avg_tens']
real_irrigation = df['sum_irr']

df['sum_irr'] = 0.0
df = df.drop(columns=['sensor_id', 'avg_tens']).sort_index(axis=1)

df

,avg_tens_lag1,avg_tens_lag2,avg_tens_lag3,max_temp,max_temp_lag1,max_temp_lag2,max_temp_lag3,sum_irr,sum_irr_lag1,sum_irr_lag2,sum_irr_lag3,sum_rain,sum_rain_lag1,sum_rain_lag2,sum_rain_lag3
date,,,,,,,,,,,,,,,
2023-05-01,128.739130,119.761905,109.759494,20.100000,23.366667,23.650000,20.066667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-05-02,137.312500,128.739130,119.761905,19.466667,20.100000,23.366667,23.650000,0.0,0.0,0.000000,0.000000,1.853333,0.000000,0.000000,0.000000
2023-05-03,133.062937,137.312500,128.739130,24.060000,19.466667,20.100000,23.366667,0.0,0.0,0.000000,0.000000,0.000000,1.853333,0.000000,0.000000
2023-05-04,128.028369,133.062937,137.312500,24.233333,24.060000,19.466667,20.100000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.853333,0.000000
2023-05-05,140.437908,128.028369,133.062937,25.966667,24.233333,24.060000,19.466667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.853333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-27,425.111111,346.523077,278.754717,25.580000,33.833333,35.300000,34.800000,0.0,0.0,292.341795,27.145571,3.000000,0.000000,0.000000,0.000000
2023-08-28,461.675676,425.111111,346.523077,22.233333,25.580000,33.833333,35.300000,0.0,0.0,0.000000,292.341795,10.520000,3.000000,0.000000,0.000000
2023-08-29,187.746479,461.675676,425.111111,22.433333,22.233333,25.580000,33.833333,0.0,0.0,0.000000,0.000000,1.200000,10.520000,3.000000,0.000000


In [8]:
df_dss = pd.DataFrame({
    'last_avg_tensiometer': df['avg_tens_lag1'],
    'predicted_avg_tensiometer': xgb_model.predict(df),
    'predicted_rain_amount': df['sum_rain'],
    'predicted_max_temperature': df['max_temp']
})

df_dss

,last_avg_tensiometer,predicted_avg_tensiometer,predicted_rain_amount,predicted_max_temperature
date,,,,
2023-05-01,128.739130,144.847447,0.000000,20.100000
2023-05-02,137.312500,137.679664,1.853333,19.466667
2023-05-03,133.062937,122.559161,0.000000,24.060000
2023-05-04,128.028369,136.259236,0.000000,24.233333
2023-05-05,140.437908,147.020174,0.000000,25.966667
...,...,...,...,...
2023-08-27,425.111111,348.726071,3.000000,25.580000
2023-08-28,461.675676,314.731704,10.520000,22.233333
2023-08-29,187.746479,80.474714,1.200000,22.433333


In [9]:
recommendations = apply_decision_system(dss, df_dss)
second_elements = [tup[1] for tup in recommendations]

results = []
for element in second_elements:
    results.append(reverse_categorize_irrigation(element))

In [10]:
show_figure(72, real_tens_values, real_tens_values, real_irrigation, results)

TypeError: show_figure() missing 2 required positional arguments: 'rain' and 'output_file'

In [97]:
rec_irrigation = []
rec_tens_values = []

predicted_rain_amount = 0.0
df['sum_irr_lag1'] = 0.0
df['sum_irr_lag2'] = 0.0
df['sum_irr_lag3'] = 0.0

flag = 0
for i in range(len(df) - 4):
    
    if i + 2 < len(df):
        predicted_rain_amount = df['sum_rain'].iloc[i] + df['sum_rain'].iloc[i + 1] + df['sum_rain'].iloc[i + 2]
    elif i + 1 < len(df):
        predicted_rain_amount = df['sum_rain'].iloc[i] + df['sum_rain'].iloc[i + 1]
    else:
        predicted_rain_amount = df['sum_rain'].iloc[i]    
   
    new_df = pd.DataFrame({
        'last_avg_tensiometer': df['avg_tens_lag1'].iloc[i:i+1],
        'predicted_avg_tensiometer': xgb_model.predict(df.iloc[i:i+1]),
        'predicted_rain_amount': predicted_rain_amount,
        'predicted_max_temperature': df['max_temp'].iloc[i:i+1]
    })


    decision = apply_decision_system(dss, new_df.iloc[0:1])[0][1]

    if decision != 'Not Recommended':
        irrigation_value = reverse_categorize_irrigation(decision)
        flag = 1
    else:
        irrigation_value = 0.0

    df['sum_irr'].iloc[i:i+1] = irrigation_value
    rec_irrigation.append(irrigation_value)
    
    df['sum_irr_lag1'].iloc[i+1:i+2] = irrigation_value
    df['sum_irr_lag2'].iloc[i+2:i+3] = irrigation_value
    df['sum_irr_lag3'].iloc[i+3:i+4] = irrigation_value

    predicted_tens = xgb_model.predict(df.iloc[i:i+1])
    rec_tens_values.append(predicted_tens[0])
    
    if decision != 'Not Recommended' or flag == 1:
        df['avg_tens_lag1'].iloc[i+1:i+2] = predicted_tens
        df['avg_tens_lag2'].iloc[i+2:i+3] = predicted_tens
        df['avg_tens_lag3'].iloc[i+3:i+4] = predicted_tens

In [98]:
show_figure(67, real_tens_values, rec_tens_values, real_irrigation, rec_irrigation)

In [99]:
real_count = sum(1 for value in real_irrigation if value > 0.0)
real_sum = sum(real_irrigation)
real_tens_days = sum(1 for value in real_tens_values if value > 400.0)

rec_count = sum(1 for value in rec_irrigation if value > 0.0)
rec_sum = sum(rec_irrigation)
rec_tens_days = sum(1 for value in rec_tens_values if value > 400.0)

# Crea una tabella
print("{:<20} {:<20} {:<20} {:<20}".format('', 'Over Threshold Days', 'Irrigation Days', 'Total Water (Liters)'))
print("-" * 60)
print("{:<20} {:<20} {:<20} {:<20}".format('Ground Truth', real_tens_days, real_count, int(real_sum)))
print("{:<20} {:<20} {:<20} {:<20}".format('Recommended', rec_tens_days, rec_count, int(rec_sum)))

                     Over Threshold Days  Irrigation Days      Total Water (Liters)
------------------------------------------------------------
Ground Truth         9                    20                   4479                
Recommended          2                    3                    975                 


In [1]:
def process_irrigation_data(file_path, xgb_model, dss, sensor_id, start_date, end_date):

    df = pd.read_csv(file_path)
    df = df[(df['sensor_id'] == sensor_id) & (df['date'] >= start_date) & (df['date'] <= end_date)]
    df.set_index('date', inplace=True)

    real_tens_values = df['avg_tens']
    real_irrigation = df['sum_irr']

    df['sum_irr'] = 0.0
    df = df.drop(columns=['sensor_id', 'avg_tens']).sort_index(axis=1)

    rec_irrigation = []
    rec_tens_values = []

    df['sum_irr_lag1'] = 0.0
    df['sum_irr_lag2'] = 0.0
    df['sum_irr_lag3'] = 0.0

    flag = 0
    for i in range(len(df) - 4):
        # Calcolo della previsione della pioggia cumulativa
        if i + 2 < len(df):
            predicted_rain_amount = df['sum_rain'].iloc[i] + df['sum_rain'].iloc[i + 1] + df['sum_rain'].iloc[i + 2]
        elif i + 1 < len(df):
            predicted_rain_amount = df['sum_rain'].iloc[i] + df['sum_rain'].iloc[i + 1]
        else:
            predicted_rain_amount = df['sum_rain'].iloc[i]

        # Creazione del nuovo dataframe per il sistema decisionale
        new_df = pd.DataFrame({
            'last_avg_tensiometer': df['avg_tens_lag1'].iloc[i:i+1],
            'predicted_avg_tensiometer': xgb_model.predict(df.iloc[i:i+1]),
            'predicted_rain_amount': predicted_rain_amount,
            'predicted_max_temperature': df['max_temp'].iloc[i:i+1]
        })

        # Applicazione del sistema decisionale DSS
        decision = apply_decision_system(dss, new_df.iloc[0:1])[0][1]

        # Calcolo del valore di irrigazione
        if decision != 'Not Recommended':
            irrigation_value = reverse_categorize_irrigation(decision)
            flag = 1
        else:
            irrigation_value = 0.0

        # Aggiornamento del dataframe e dei valori raccomandati
        df['sum_irr'].iloc[i:i+1] = irrigation_value
        rec_irrigation.append(irrigation_value)

        df['sum_irr_lag1'].iloc[i+1:i+2] = irrigation_value
        df['sum_irr_lag2'].iloc[i+2:i+3] = irrigation_value
        df['sum_irr_lag3'].iloc[i+3:i+4] = irrigation_value

        predicted_tens = xgb_model.predict(df.iloc[i:i+1])
        rec_tens_values.append(predicted_tens[0])

        if decision != 'Not Recommended' or flag == 1:
            df['avg_tens_lag1'].iloc[i+1:i+2] = predicted_tens
            df['avg_tens_lag2'].iloc[i+2:i+3] = predicted_tens
            df['avg_tens_lag3'].iloc[i+3:i+4] = predicted_tens

    # Calcolo delle statistiche richieste
    real_count = sum(1 for value in real_irrigation if value > 0.0)
    real_sum = sum(real_irrigation)
    real_tens_days = sum(1 for value in real_tens_values if value > 400.0)

    rec_count = sum(1 for value in rec_irrigation if value > 0.0)
    rec_sum = sum(rec_irrigation)
    rec_tens_days = sum(1 for value in rec_tens_values if value > 400.0)

    return real_count, real_sum, real_tens_days, rec_count, rec_sum, rec_tens_days

In [2]:
real_count, real_sum, real_tens_days, rec_count, rec_sum, rec_tens_days = process_irrigation_data('../data/dss_data_rovere.csv', xgb_model, dss, 67, '2023-05-01', '2023-08-31')    

NameError: name 'xgb_model' is not defined

In [ ]:
print("{:<20} {:<20} {:<20} {:<20}".format('', 'Over Threshold Days', 'Irrigation Days', 'Total Water (Liters)'))
print("-" * 60)
print("{:<20} {:<20} {:<20} {:<20}".format('Ground Truth', real_tens_days, real_count, int(real_sum)))
print("{:<20} {:<20} {:<20} {:<20}".format('Recommended', rec_tens_days, rec_count, int(rec_sum)))

In [19]:
real_count, real_sum, real_tens_days, rec_count, rec_sum, rec_tens_days = process_irrigation_data('../data/dss_data_rovere.csv', xgb_model, dss, 61, '2023-05-01', '2023-08-31')

In [20]:
print("{:<20} {:<20} {:<20} {:<20}".format('', 'Over Threshold Days', 'Irrigation Days', 'Total Water (Liters)'))
print("-" * 60)
print("{:<20} {:<20} {:<20} {:<20}".format('Ground Truth', real_tens_days, real_count, int(real_sum)))
print("{:<20} {:<20} {:<20} {:<20}".format('Recommended', rec_tens_days, rec_count, int(rec_sum)))

                     Over Threshold Days  Irrigation Days      Total Water (Liters)
------------------------------------------------------------
Ground Truth         29                   21                   23757               
Recommended          16                   14                   5200                
